In [ ]:
import os
import cv2
import glob
from PIL import Image
import  matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import sys
sys.path.append('/kaggle/input/monai-019/')

import shutil
shutil.copy('/kaggle/input/eeg2spectrogram/preprocessing.py','preprocessing.py')
from preprocessing import spectrogram_from_eeg

In [ ]:
class Slide_Window_Model(nn.Module):
    def __init__(self, model_name, cls):
        super().__init__()
        
    def forward(self, image):
        x = self.model(image)
        return x.view(-1, self.cls, 1, 1)

In [ ]:
def get_test_transform(img_size):
    return A.Compose([
        A.PadIfNeeded(min_height=400, min_width=CFG['img_crop'], border_mode=0, p=1),
        ToTensorV2(p=1.0),
    ])

class Customize_Dataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms
        
    def read_data(self, data):
        def norm_to_255(img):
            img= img-img.min()
            img= img/img.max()
            img= img*255
            return img.astype(np.uint8)
        
        def norm_to_standard(img):
            ep = 1e-6
            m = np.nanmean(img.flatten())
            s = np.nanstd(img.flatten())
            img = (img-m)/(s+ep)
            img = np.nan_to_num(img, nan=0.0)
            return img
        
        raw= pd.read_parquet(data['spectrogram_path']).fillna(0)
        
        col= list(raw.filter(like='LL', axis=1))
        img_LL= np.log1p(raw[col].T.values)
        col= list(raw.filter(like='RL', axis=1))
        img_RL= np.log1p(raw[col].T.values)
        col= list(raw.filter(like='RP', axis=1))
        img_RP= np.log1p(raw[col].T.values)
        col= list(raw.filter(like='LP', axis=1))
        img_LP= np.log1p(raw[col].T.values)
        
        img= np.concatenate([img_LL, img_LP, img_RP, img_RL], axis=0)
        img= np.expand_dims(img, axis=2)
        img= np.concatenate([img, img, img], axis=2)
        img_spectrograms= norm_to_standard(img)
        
        ## test_eegs
        img_10= spectrogram_from_eeg(data['eeg_path'], duration=10, height=100)
        img_10= np.concatenate([img_10[..., 0],
                                img_10[..., 1],
                                img_10[..., 2],
                                img_10[..., 3]], axis=0)
        img_30= spectrogram_from_eeg(data['eeg_path'], duration=30, height=100)
        img_30= np.concatenate([img_30[..., 0],
                                img_30[..., 1],
                                img_30[..., 2],
                                img_30[..., 3]], axis=0)
        img= np.concatenate([img_10, img_30], axis=1)
        img= np.expand_dims(img, axis=2)
        img= np.concatenate([img, img, img], axis=2)
        img_eeg= img
        
        ## fuse img
        img_spectrograms= img_spectrograms[:, :, :1]
        img_eeg= img_eeg[..., :1]
        img= np.concatenate([img_eeg, img_spectrograms], axis=1)
        img= np.concatenate([img, img, img], axis=2)
        
        return img
    
    def __getitem__(self, index):
        data = self.df.loc[index]
        img= self.read_data(data)
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
            
        return {
            'image': torch.tensor(img, dtype=torch.float32),
        }
    
    def __len__(self):
        return len(self.df)

## CFG

In [ ]:
CFG= {
    'img_size': None,
    'img_crop': 912,
    'TTA': 1,
    
    'model': [
        '/kaggle/input/effb0-lb33',
        '/kaggle/input/effb0-kd-lb32',
        '/kaggle/input/cvnxt-kd-lb32',
        '/kaggle/input/effv2s-kd-lb31',
        
        '/kaggle/input/effb0-1',
    ],
    'model_weight': [
        0.1,
        0.1,
        0.1,
        0.2,
        
        0.5,
    ],
}

## load model
Models= []
for i in range(len(CFG['model'])):
    models= []
    for m in glob.glob(CFG['model'][i]+'/**'):
        models.append( torch.load(m, map_location= 'cuda:0') )
    Models.append(models)
CFG['model']= Models
print(f"length of model: {len(Models)}")

## Prepare Dataset

In [ ]:
test_df= pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
spectrogram_path= '/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms'
eeg_path= '/kaggle/input/hms-harmful-brain-activity-classification/test_eegs'
for i in range(len(test_df)):
    test_df.loc[i,'spectrogram_path']= f"{spectrogram_path}/{test_df.loc[i,'spectrogram_id']}.parquet"
    test_df.loc[i,'eeg_path']= f"{eeg_path}/{test_df.loc[i,'eeg_id']}.parquet"
    
test_dataset= Customize_Dataset(test_df, get_test_transform(CFG['img_size']))
test_loader= DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)
test_df.head()

## Inference

In [ ]:
from monai.inferers import sliding_window_inference

def sliding_inference(model, img):
    
    img= torch.unsqueeze(img, 0).cuda()
    for i, m in enumerate(model):
        with torch.no_grad():
            m.eval()
            
            imgs= torch.cat([img, img.flip(-1), img.flip(-2), img.flip(-1).flip(-2)], dim=0)
            with torch.no_grad():
                pred= sliding_window_inference(imgs[:CFG['TTA']],
                                                roi_size=(-1,CFG['img_crop']), 
                                                mode= 'gaussian',
                                                sw_batch_size=4, 
                                                predictor=m)
                pred= pred.view(pred.shape[0],6,-1).mean(dim=-1)
                pred= pred.mean(dim=0)
            
        if i==0: preds= pred.softmax(dim=-1)
        else: preds+= pred.softmax(dim=-1)
            
    preds= preds/len(model)
    preds= preds.cpu().numpy()
    return preds

In [ ]:
count= 0
for i, data in enumerate(tqdm(test_loader)):
    for j in range(len(data['image'])):
        img= data['image'][j]
        if count==0: print(img.shape)
            
        ## Model Inference
        for k,m in enumerate(CFG['model']):
            prob= sliding_inference(m, img)
            
            if k==0: pred= prob * CFG['model_weight'][k]
            else: pred+= prob * CFG['model_weight'][k]
                
        test_df.loc[count, 'pred_cls']= pred.argmax(0)
        test_df.loc[count, 'prob']= str(pred.tolist())
        count+= 1
test_df.head()

## Submission

In [ ]:
submit= pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv')
for i in range(len(submit)):
    pred= eval(test_df.loc[i, 'prob'])
    submit.loc[i, submit.columns[1:] ]= pred
    
submit.to_csv('submission_richard_lb31.csv',index=False)
submit